In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
file = "SpeciesConc"
normal_file = f"/home/ucfacb0/rundirs/Rockets/gc_4x5_47L_merra2_fullchem_aluminachemtesting/OutputDir/norrtmg/normalchem_aermassout/GEOSChem.{file}.20200101_0000z.nc4"
alumina_file = f"/home/ucfacb0/rundirs/Rockets/gc_4x5_47L_merra2_fullchem_aluminachemtesting/OutputDir/norrtmg/gravsettlechem3/GEOSChem.{file}.20200101_0000z.nc4"
ds_alumina = xr.open_dataset(alumina_file)
ds_normal = xr.open_dataset(normal_file)
ds_normal

<xarray.Dataset>
Dimensions:                 (time: 1, lev: 47, ilev: 48, lat: 46, nb: 2, lon: 72)
Coordinates:
  * time                    (time) datetime64[ns] 2020-01-01
  * lev                     (lev) float64 0.9925 0.9775 ... 0.0001387 3.8e-05
  * ilev                    (ilev) float64 1.0 0.985 0.97 ... 6.6e-05 1e-05
  * lat                     (lat) float64 -89.0 -86.0 -82.0 ... 82.0 86.0 89.0
  * lon                     (lon) float64 -180.0 -175.0 -170.0 ... 170.0 175.0
Dimensions without coordinates: nb
Data variables: (12/319)
    lat_bnds                (lat, nb) float64 ...
    lon_bnds                (lon, nb) float64 ...
    hyam                    (lev) float64 ...
    hybm                    (lev) float64 ...
    hyai                    (ilev) float64 ...
    hybi                    (ilev) float64 ...
    ...                      ...
    SpeciesConcVV_AONITA    (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ALK4      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ALD2      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_AERI      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ACTA      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ACET      (time, lev, lat, lon) float32 ...
Attributes:
    title:                           GEOS-Chem diagnostic collection: Species...
    history:                         
    format:                          NetCDF-4
    conventions:                     COARDS
    ProdDateTime:                    
    reference:                       www.geos-chem.org; wiki.geos-chem.org
    contact:                         GEOS-Chem Support Team (geos-chem-suppor...
    simulation_start_date_and_time:  2020-01-01 00:00:00z
    simulation_end_date_and_time:    2020-01-02 00:00:00z

In [2]:
varnames = list(ds_normal.data_vars.keys())
varname_list, normal_list, alumina_list, diff_list, diffpercent_list = [],[],[],[],[]
for varname in varnames:
    varname_list.append(varname)
    normal_list.append(np.sum(ds_normal[varname].values))
    alumina_list.append(np.sum(ds_alumina[varname].values))
    diff_list.append(np.abs(np.sum(ds_alumina[varname].values)-np.sum(ds_normal[varname].values)))
    diffpercent_list.append((np.abs(np.sum(ds_alumina[varname].values)-np.sum(ds_normal[varname].values)))/(np.sum(ds_normal[varname].values)*100))

data = pd.DataFrame({'varname': varname_list, 'normal_list': normal_list, 'alumina_list': alumina_list, 'diff_list': diff_list, 'diffpercent_list': diffpercent_list},
                    columns=['varname', 'normal_list', 'alumina_list', 'diff_list','diffpercent_list'])

def make_pretty(styler):
    styler.background_gradient(axis=1, vmin=0, vmax=1, cmap="YlGnBu",subset="diffpercent_list")
    styler.format('{:.4e}', subset=['normal_list', 'alumina_list', 'diff_list'])
    styler.format('{:.4e}', subset=['diffpercent_list'])
    return styler


#data.to_csv(f"{file}.csv",sep=',')
#data = data.sort_values(["diffpercent_list"], ascending=False)
data[data["diff_list"] != None].style.pipe(make_pretty)


/tmp/ipykernel_196676/1450681563.py:8: RuntimeWarning: invalid value encountered in scalar divide
  diffpercent_list.append((np.abs(np.sum(ds_alumina[varname].values)-np.sum(ds_normal[varname].values)))/(np.sum(ds_normal[varname].values)*100))


,varname,normal_list,alumina_list,diff_list,diffpercent_list
0,lat_bnds,0.0000e+00,0.0000e+00,0.0000e+00,nan
1,lon_bnds,-3.6000e+02,-3.6000e+02,0.0000e+00,-0.0000e+00
2,hyam,4.2906e+03,4.2906e+03,0.0000e+00,0.0000e+00
3,hybm,1.7488e+01,1.7488e+01,0.0000e+00,0.0000e+00
4,hyai,4.2906e+03,4.2906e+03,0.0000e+00,0.0000e+00
5,hybi,1.7988e+01,1.7988e+01,0.0000e+00,0.0000e+00
6,P0,1.0000e+03,1.0000e+03,0.0000e+00,0.0000e+00
7,AREA,5.1007e+14,5.1007e+14,0.0000e+00,0.0000e+00
8,SpeciesConcVV_RCOOH,1.4516e-15,1.4516e-15,0.0000e+00,0.0000e+00
9,SpeciesConcVV_O2,3.0410e+04,3.0410e+04,0.0000e+00,0.0000e+00


In [3]:
#aeroraditot_normal, aeroraditot_alumina = 0,0
#for i in range(3,8):
#    aeroraditot_normal += np.sum(ds_normal[f"Chem_AeroAreaMDUST{i}"].values)
#    aeroraditot_alumina += np.sum(ds_alumina[f"Chem_AeroAreaMDUST{i}"].values)
#print(aeroraditot_normal,aeroraditot_alumina)

In [4]:
varnames = list(ds_alumina.data_vars.keys())
varname_list, normal_list, alumina_list, diff_list = [],[],[],[]
for varname in varnames:
    varname_list.append(varname)
    alumina_list.append(np.sum(ds_alumina[varname].values))

data = pd.DataFrame({'varname': varname_list, 'alumina_list': alumina_list},
                    columns=['varname', 'alumina_list'])

def make_pretty(styler):
    styler.background_gradient(axis=1, vmin=0, vmax=200, cmap="YlGnBu",subset="alumina_list")
    styler.format('{:.2f}', subset=['alumina_list'])
    return styler

data[data["varname"].str.contains("AL2O3|ALU")].style.pipe(make_pretty)

,varname,alumina_list
316,SpeciesConcVV_AL2O3,0.00


In [5]:
ds_alumina.close()
ds_normal.close()